In [2]:
import pandas as pd
import os

The code below is used to determine whether the crime is categorized as `personal` or `property`.

In [2]:
# crime = pd.read_csv('data/Crime_zip/crime_data_2020_zip_final.csv', index_col=0)

# personal = [110, 113, 121, 122, 
#             210, 220, 230, 231, 235, 236, 237, 250, 251, 
#             434, 435, 436, 437, 439, 
#             622, 623, 624, 625, 626, 627, 647, 651, 652, 653, 654, 661, 
#             753, 755, 756, 760, 761, 762, 763, 
#             805, 806, 810, 812, 813, 814, 815, 820, 821, 822, 845, 850, 860, 865, 870, 
#             900, 901, 902, 903, 906, 910, 920, 921, 922, 926, 928, 930, 932, 940, 942, 943, 944, 946, 954, 956]

# property = [310, 320, 330, 331, 341, 343, 345, 347, 349, 350, 351, 352, 353, 354, 
#             410, 420, 421, 433, 438, 440, 441, 442, 443, 444, 445, 446, 450, 
#             451, 452, 453, 470, 471, 472, 473, 474, 475, 480, 485, 487, 491, 
#             510, 520, 522, 648, 649, 660, 662, 664, 666, 668, 670, 
#             740, 745, 880, 886, 888, 890, 
#             931, 933, 949, 950, 951]

# def crime_type(crime_code):
#     if crime_code in personal:
#         return 'personal'
#     else:
#         return 'property'
    
# crime['Crime_Type'] = crime['Crm Cd'].apply(crime_type)

# crime_counts = crime.groupby(['ZIP', 'Crime_Type']).size().reset_index(name='Crime_Count')

Since I can't upload the whole `crime_data_2020_zip_final` dataset, you can use the `crime_count` file, which I created using the code above.

In [3]:
crime_counts = pd.read_csv('crime_count.csv', index_col=0)
crime_counts

,ZIP,Crime_Type,Crime_Count
0,90001,personal,340
1,90001,property,354
2,90002,personal,1261
3,90002,property,1171
4,90003,personal,3335
...,...,...,...
338,91606,property,1232
339,91607,personal,285
340,91607,property,762
341,91608,personal,56


In [4]:
popu = pd.read_csv('DECENNIALDHC2020.P1-Data.csv', skiprows=[1])
popu = popu.rename(columns={"P1_001N": "Population"})

In [5]:
def get_zip(in_file):
    for index, row in in_file.iterrows():
        zip_original = row['NAME']
        zip_new = int(zip_original[-5:])

        in_file.at[index, 'ZIP'] = zip_new

In [6]:
get_zip(popu)
popu

,GEO_ID,NAME,Population,P1_001NA,ZIP
0,860Z200US89010,ZCTA5 89010,333,NaN,89010.0
1,860Z200US89019,ZCTA5 89019,2501,NaN,89019.0
2,860Z200US89060,ZCTA5 89060,11857,NaN,89060.0
3,860Z200US89061,ZCTA5 89061,7891,NaN,89061.0
4,860Z200US89439,ZCTA5 89439,2154,NaN,89439.0
...,...,...,...,...,...
1803,860Z200US96148,ZCTA5 96148,993,NaN,96148.0
1804,860Z200US96150,ZCTA5 96150,29518,NaN,96150.0
1805,860Z200US96155,ZCTA5 96155,8,NaN,96155.0
1806,860Z200US96161,ZCTA5 96161,19651,NaN,96161.0


In [7]:
result_df = crime_counts.merge(popu, on='ZIP', how='left')
result_df = result_df[['ZIP', 'Crime_Type', 'Crime_Count', 'Population']]
result_df['Population'] = result_df['Population'].fillna(0)
result_df

,ZIP,Crime_Type,Crime_Count,Population
0,90001,personal,340,55859.0
1,90001,property,354,55859.0
2,90002,personal,1261,53150.0
3,90002,property,1171,53150.0
4,90003,personal,3335,72764.0
...,...,...,...,...
338,91606,property,1232,43955.0
339,91607,personal,285,30502.0
340,91607,property,762,30502.0
341,91608,personal,56,37.0


In [8]:
def calculate_rate(row):
    if row['Population'] > 0:
        rate = row['Crime_Count'] / row['Population']
    else:
        rate = 1
    
    return rate

In [9]:
result_df['Crime_Rate'] = result_df.apply(calculate_rate, axis=1)
result_df

,ZIP,Crime_Type,Crime_Count,Population,Crime_Rate
0,90001,personal,340,55859.0,0.006087
1,90001,property,354,55859.0,0.006337
2,90002,personal,1261,53150.0,0.023725
3,90002,property,1171,53150.0,0.022032
4,90003,personal,3335,72764.0,0.045833
...,...,...,...,...,...
338,91606,property,1232,43955.0,0.028029
339,91607,personal,285,30502.0,0.009344
340,91607,property,762,30502.0,0.024982
341,91608,personal,56,37.0,1.513514


In [10]:
output_directory = 'data/'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
result_df.to_csv('data/crime_rate_2020.csv')